# Group 7 Lab 2

***
setting up graph

In [1]:
# some code

***
a) Suppose the input graph G = (V, E) is stored in an adjacency matrix and we 
use an array for the priority queue. Implement the Dijkstra’s algorithm using this 
setting and analyze its time complexity with respect to |V| and |E| both 
theoretically and empirically.

In [ ]:
# some code

***
b) Suppose the input graph G = (V, E) is stored in an array of adjacency lists and 
we use a minimizing heap for the priority queue. Implement the Dijkstra’s 
algorithm using this setting and analyze its time complexity with respect to |V| 
and |E| both theoretically and empirically. 

In [ ]:
# some code

***
c) Compare the two implementations in (a) and (b). Discuss which implementation 
is better and in what circumstances. 

In [ ]:
# some code

***
conclusion